# Neural Networks & Deep Learning Project. 
## Done by : Yassine LAHBABI - DIA 4

## BackPropagation :

We will implement a backpropogration algorithm for a neural network from scratch. It ca be used for both classification and regression problems, we will focus on classification in this exercise. 

### Importing the modules that we need : 

In [2]:
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.metrics import mean_squared_error 
import matplotlib.pyplot as plt
import numpy as np

### Initialize Network : 

#### Ex 1 :

In [248]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

#### Ex 2 : Testing our function. 

In [249]:
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
	print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


### Forward Propagate : 

#### Ex 3 : Neuron Activation. 

In [250]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

#### Ex 4 : Neuron Transfer. 

In [251]:
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))

 #### Ex 5 : Forward Propagation.

In [252]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
	inputs = new_inputs
	return inputs

#### Ex 6 : Testing the forward Propagation. 

In [253]:
# test forward propagation
network = initialize_network(3,2,2)
row = [1, 0, 0]
output = forward_propagate(network, row)
print(output)

[0.8411501450335089, 0.7164405193083182]


### Back Propagate Error : 

#### Ex 7 : Transfer derivative using the sigmoid.

In [254]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

#### Ex 8 : Error Backpropagation.

In [255]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

#### Ex 9 : Testing the backward propagate.

In [256]:
# test backpropagation of error
network = initialize_network(3,2,2)
expected = [0, 1]
forward_propagate(network,[1,1,0])
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'weights': [0.5414124727934966, 0.9391491627785106, 0.38120423768821243, 0.21659939713061338], 'output': 0.8451635856379899, 'delta': -0.007822193728794707}, {'weights': [0.4221165755827173, 0.029040787574867943, 0.22169166627303505, 0.43788759365057206], 'output': 0.7086930458657226, 'delta': -0.0017572370152870301}]
[{'weights': [0.49581224138185065, 0.23308445025757263, 0.2308665415409843], 'output': 0.723074397958357, 'delta': -0.14478683606576973}, {'weights': [0.2187810373376886, 0.4596034657377336, 0.28978161459048557], 'output': 0.7247538152739895, 'delta': 0.05490768403088252}]


### Training the network : 

#### Ex 10 : Updating Weights. 

In [257]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

#### Ex 11 : Train Network. 

In [258]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

#### Ex 12 : Small dataset to test out training our neural network.

In [259]:
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.500, error=9.723
>epoch=1, lrate=0.500, error=9.771
>epoch=2, lrate=0.500, error=9.822
>epoch=3, lrate=0.500, error=9.865
>epoch=4, lrate=0.500, error=9.903
>epoch=5, lrate=0.500, error=9.939
>epoch=6, lrate=0.500, error=9.979
>epoch=7, lrate=0.500, error=10.024
>epoch=8, lrate=0.500, error=10.076
>epoch=9, lrate=0.500, error=10.137
>epoch=10, lrate=0.500, error=10.208
>epoch=11, lrate=0.500, error=10.289
>epoch=12, lrate=0.500, error=10.378
>epoch=13, lrate=0.500, error=10.474
>epoch=14, lrate=0.500, error=10.571
>epoch=15, lrate=0.500, error=10.668
>epoch=16, lrate=0.500, error=10.763
>epoch=17, lrate=0.500, error=10.854
>epoch=18, lrate=0.500, error=10.935
>epoch=19, lrate=0.500, error=11.001
[{'weights': [-0.6125991588575579, 0.782444115821657, 0.6963163161846515], 'output': 0.22347212273536932, 'delta': -0.0003828462540796415}, {'weights': [0.49035921398346644, -0.2515673358120472, 0.18617056549654923], 'output': 0.9550920970840151, 'delta': 0.0002557995665174358

### Predict : 

#### Ex 13 : Predict function.

In [260]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

#### Ex 14 : Testing the predict function. 

In [270]:
# Test making predictions with the network
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
network = [[{'weights': [-1.482313569067226, 1.8308790073202204, 1.078381922048799]}, {'weights': [0.23244990332399884, 0.3621998343835864, 0.40289821191094327]}],
	[{'weights': [2.5001872433501404, 0.7887233511355132, -1.1026649757805829]}, {'weights': [-2.429350576245497, 0.8357651039198697, 1.0699217181280656]}]]
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


### Seeds Dataset : 

Now, we will apply what we did earlier to our Seeds Dataset. 
We will apply the Backpopagation algorithm to the Seeds Dataset by loading the dataset and then applying our functions.


#### Ex 15 : Downloading the seeds_dataset.csv

#### Ex 16 : Loading the dataset. 

In [3]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

#### Ex 17 : Preparing the Dataset.

In [8]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

#### Ex 18 : Initializing a network for our dataset and modifying our previous functions for our dataset. 

In [9]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation
 
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))
 
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs
 
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
 
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']
 
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
 
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network
 
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0


# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
 

 

#### Ex 19 : Predicting the test set on trained network & computing the MSE. 

In [10]:
with open("seeds_dataset.csv", "r") as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace("\t", ",")
filedata = filedata.replace("\"", "")

# Write the file out again
with open("exit.csv", "w") as file:
  file.write(filedata)


# Test Backprop on Seeds dataset
seed(1)
dataset = load_csv('exit.csv')
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
mean_error = (1-((sum(scores)/float(len(scores)))/100))**2
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
print('Mean Squared Error', mean_error)

Scores: [92.85714285714286, 92.85714285714286, 97.61904761904762, 92.85714285714286, 90.47619047619048]
Mean Accuracy: 93.333%
Mean Squared Error 0.004444444444444428
